In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

In [16]:
def rsi_14_strategy(data):
    data = data.copy() 
    data['Change'] = data['Close'].diff()

    data['Gain'] = data['Change'].where(data['Change'] > 0, 0)
    data['Loss'] = -data['Change'].where(data['Change'] < 0, 0)
    
    data['Average Gain'] = np.where(data.index < data.index[14], data['Gain'].rolling(window=14).mean(), np.nan)
    data['Average Loss'] = np.where(data.index < data.index[14], data['Loss'].rolling(window=14).mean(), np.nan)

    for i in range(14, len(data)):
        data['Average Gain'].iloc[i] = (data['Average Gain'].iloc[i-1] * 13 + data['Gain'].iloc[i]) / 14
        data['Average Loss'].iloc[i] = (data['Average Loss'].iloc[i-1] * 13 + data['Loss'].iloc[i]) / 14

    data['RS'] = data['Average Gain'] / data['Average Loss']

    data['RSI'] = (100 - (100/(1+data['RS'])))
    data['RSI_action'] = np.where(data['RSI'] < 50, 1, np.where(data['RSI'] >= 50, -1,0))
    
    return data[['RSI','RSI_action']]


In [17]:
def ema(data, window):
    data = data.copy()
    
    first_valid_index = data['Close'].first_valid_index()
    if first_valid_index is not None:
        initial_sma_position = data.index.get_loc(first_valid_index) + window - 1
        if initial_sma_position < len(data):
            initial_sma = data['Close'].rolling(window=window).mean().iloc[initial_sma_position]
        else:
            initial_sma = np.nan
    else:
        initial_sma = np.nan

    multiplier = (2 / (window + 1))
    
    data[f'{window}ema'] = np.nan
    
    if not np.isnan(initial_sma):
        data[f'{window}ema'].iloc[initial_sma_position] = initial_sma
    
        for i in range(initial_sma_position + 1, len(data)):
            data[f'{window}ema'].iloc[i] = (data['Close'].iloc[i] - data[f'{window}ema'].iloc[i-1]) * multiplier + data[f'{window}ema'].iloc[i-1]
    
    return data[[f'{window}ema']]


In [18]:
def ema_strategy(data):
    data = aapl.copy()
    ema_line = ema(data,9).copy()
        
    conditions = [
        (data['Close'] > ema_line['9ema']) & (data['Close'].notna()) & (ema_line['9ema'].notna()),
        (data['Close'] < ema_line['9ema']) & (data['Close'].notna()) & (ema_line['9ema'].notna())
    ]

    choices = [1, -1]

    data['ema_strategy_action'] = np.select(conditions, choices, default=0)

    return data[['ema_strategy_action']]

In [19]:
def macd_strategy(data):
    data = data.copy()

    fastMA = ema(data, 12)
    slowMA = ema(data, 26)
    data['macd'] = fastMA['12ema'] - slowMA['26ema']

    data['macd_action'] = np.where(data['macd'] < 0, -1, np.where(data['macd'] > 0, 1,0))

    signal_data = data[['macd']].rename(columns={'macd': 'Close'})
    signal_line = ema(signal_data, 9)

    data['signal_line'] = signal_line['9ema']
    data['macd_strategy_action'] = np.where(data['macd'] > data['signal_line'], 1, np.where(data['macd'] < data['signal_line'], -1,0))
    
    return data[['macd','macd_action','signal_line','macd_strategy_action']]


In [20]:
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META']
all_results = {}

for ticker in tickers:
    data = yf.download(ticker, start='2010-01-01',)[['Close']]
    results = data.copy()

    ema_10 = ema(data, 10)
    ema_strat = ema_strategy(data)
    macd = macd_strategy(data)
    rsi = rsi_14_strategy(data)
    

    results = results.join([ema_10, ema_strat, macd, rsi])
    all_results[ticker] = results

all_results

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


{'AAPL':                  Close       10ema  ema_strategy_action      macd  \
 Date                                                                
 2010-01-04    7.643214         NaN                  NaN       NaN   
 2010-01-05    7.656429         NaN                  NaN       NaN   
 2010-01-06    7.534643         NaN                  NaN       NaN   
 2010-01-07    7.520714         NaN                  NaN       NaN   
 2010-01-08    7.570714         NaN                  NaN       NaN   
 ...                ...         ...                  ...       ...   
 2023-10-09  178.990005  175.209149                  1.0 -1.165012   
 2023-10-10  178.389999  175.787485                  1.0 -0.821506   
 2023-10-11  179.800003  176.517034                  1.0 -0.430537   
 2023-10-12  180.710007  177.279393                  1.0 -0.046723   
 2023-10-13  178.850006  177.564959                  1.0  0.106143   
 
             macd_action  signal_line  macd_strategy_action        RSI  \
 Date 

In [21]:
all_results_df = pd.concat(all_results, axis=1)
all_results_df

AAPL                                                        \
                 Close       10ema ema_strategy_action      macd macd_action   
Date                                                                           
2010-01-04    7.643214         NaN                 NaN       NaN           0   
2010-01-05    7.656429         NaN                 NaN       NaN           0   
2010-01-06    7.534643         NaN                 NaN       NaN           0   
2010-01-07    7.520714         NaN                 NaN       NaN           0   
2010-01-08    7.570714         NaN                 NaN       NaN           0   
...                ...         ...                 ...       ...         ...   
2023-10-09  178.990005  175.209149                 1.0 -1.165012          -1   
2023-10-10  178.389999  175.787485                 1.0 -0.821506          -1   
2023-10-11  179.800003  176.517034                 1.0 -0.430537          -1   
2023-10-12  180.710007  177.279393                 1.0 -0.046723          -1   
2023-10-13  178.850006  177.564959                 1.0  0.106143           1   

                                                                         MSFT  \
           signal_line macd_strategy_action        RSI RSI_action       Close   
Date                                                                            
2010-01-04         NaN                    0        NaN          0   30.950001   
2010-01-05         NaN                    0        NaN          0   30.959999   
2010-01-06         NaN                    0        NaN          0   30.770000   
2010-01-07         NaN                    0        NaN          0   30.450001   
2010-01-08         NaN                    0        NaN          0   30.660000   
...                ...                  ...        ...        ...         ...   
2023-10-09   -1.977853                    1  55.344892         -1  329.820007   
2023-10-10   -1.746583                    1  53.987179         -1  328.390015   
2023-10-11   -1.483374                    1  56.676854         -1  332.420013   
2023-10-12   -1.196044                    1  58.368279         -1  331.160004   
2023-10-13   -0.935607                    1  53.749160         -1  327.730011   

            ...      GOOGL        META                                  \
            ... RSI_action       Close       10ema ema_strategy_action   
Date        ...                                                          
2010-01-04  ...          0         NaN         NaN                 NaN   
2010-01-05  ...          0         NaN         NaN                 NaN   
2010-01-06  ...          0         NaN         NaN                 NaN   
2010-01-07  ...          0         NaN         NaN                 NaN   
2010-01-08  ...          0         NaN         NaN                 NaN   
...         ...        ...         ...         ...                 ...   
2023-10-09  ...         -1  318.359985  307.464395                 1.0   
2023-10-10  ...         -1  321.839996  310.078141                 1.0   
2023-10-11  ...         -1  327.820007  313.303935                 1.0   
2023-10-12  ...         -1  324.160004  315.277765                 1.0   
2023-10-13  ...         -1  314.690002  315.170899                 1.0   

                                                                              \
                macd macd_action signal_line macd_strategy_action        RSI   
Date                                                                           
2010-01-04       NaN         NaN         NaN                  NaN        NaN   
2010-01-05       NaN         NaN         NaN                  NaN        NaN   
2010-01-06       NaN         NaN         NaN                  NaN        NaN   
2010-01-07       NaN         NaN         NaN                  NaN        NaN   
2010-01-08       NaN         NaN         NaN                  NaN        NaN   
...              ...         ...         ...                  ...        ...   
2023-